In [ ]:
import os
import glob
import pandas as pd
from argparse import ArgumentParser
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import GroupKFold

In [2]:
ALL_FILENAMES = glob.glob( '../dataset/npy_data/*.npy')
train_df = pd.read_csv("../dataset/train.csv")
supp_df = pd.read_csv("../dataset/supplemental_metadata.csv")

print(len(ALL_FILENAMES))
usable_names = set()
for n in ALL_FILENAMES:
    usable_names.add(os.path.split(n)[-1])
gkf = GroupKFold(n_splits=5)

120165


In [ ]:
nonnull = []
for file,sequence_phrase in tqdm(zip(difficult.files,difficult.labels)):
    path_to_sign = f"../dataset/npy_data/{file}"


In [3]:
from signet.configs.base import Base
CFG = Base()

In [5]:
all_nan = {}
seq_length = {}
nonnullhand_length = {}
for fpth in tqdm(ALL_FILENAMES):
    npy = np.load(fpth)
    if np.isfinite(npy).mean()==1:
        all_nan[os.path.split(fpth)[-1].split(".")[0]] = 0
    name=os.path.split(fpth)[-1].split(".")[0]
    seq_length[name] = len(npy)
    nonnullhand_length[name] = (np.isnan(npy[:,CFG.LHAND+CFG.RHAND].reshape(len(npy),-1)).mean(1)!=1).sum()

100%|██████████| 120165/120165 [02:14<00:00, 892.51it/s]


In [41]:
import tensorflow as tf
def parse_example(example_proto):
    # Define the features within the example
    feature_description = {
        'coordinates': tf.io.FixedLenFeature([], tf.string),
        'sequence_id': tf.io.FixedLenFeature([], tf.int64),
        'sign': tf.io.VarLenFeature(tf.int64),
    }

    # Parse the input tf.Example proto using the dictionary above.
    parsed_example = tf.io.parse_single_example(example_proto, feature_description)

    # Decode the coordinates
    coordinates = tf.io.decode_raw(parsed_example['coordinates'], tf.float32)

    # The 'sign' feature is a variable length feature, we have to convert it from sparse to dense
    sign = tf.sparse.to_dense(parsed_example['sign'])
    
    return coordinates, parsed_example['sequence_id'], sign
def process_tfrecord_file(tfrecord_name):
    # Load the data from the TFRecord file
    raw_dataset = tf.data.TFRecordDataset(tfrecord_name, compression_type='GZIP')

    # Parse the data
    parsed_dataset = raw_dataset.map(parse_example)
    
    for coordinates, seqid, sign in parsed_dataset:
        # process data here

        # Note: To convert tensors back to numpy use `.numpy()`
        # e.g., `numpy_seqid = seqid.numpy()`
        # print(f"Coordinates: {coordinates.numpy().shape}")
        # print(f"Seqid: {seqid}")
        # print(f"Sign: {sign}")
        return coordinates.numpy().reshape(-1,543,3)

In [43]:
process_tfrecord_file("../dataset/tdf_data/1814394110.tfrecords").shape

2023-07-22 05:01:39.673076: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


(248, 543, 3)

In [6]:
import json

In [11]:
# json.dump(seq_length,open("../dataset/folds/seqlen.json","w"))
# json.dump(all_nan,open("../dataset/folds/allnan.json","w"))
json.dump({k:int(v) for k,v in nonnullhand_length.items()},open("../dataset/folds/handnotnull.json","w"))


In [12]:
train_df["seqlen"] = train_df.sequence_id.apply(lambda x: seq_length[str(x)])
train_df["nonullhand"] = train_df.sequence_id.apply(lambda x: nonnullhand_length[str(x)])
train_df["labellen"] = train_df.phrase.apply(len)

In [13]:
train_df

,path,file_id,sequence_id,participant_id,phrase,seqlen,nonullhand,labellen
0,train_landmarks/5414471.parquet,5414471,1816796431,217,3 creekhouse,123,58,12
1,train_landmarks/5414471.parquet,5414471,1816825349,107,scales/kuhaylah,127,63,15
2,train_landmarks/5414471.parquet,5414471,1816909464,1,1383 william lanier,236,178,19
3,train_landmarks/5414471.parquet,5414471,1816967051,63,988 franklin lane,102,56,17
4,train_landmarks/5414471.parquet,5414471,1817123330,89,6920 northeast 661st road,248,207,25
...,...,...,...,...,...,...,...,...
67203,train_landmarks/2118949241.parquet,2118949241,388192924,88,431-366-2913,126,35,12
67204,train_landmarks/2118949241.parquet,2118949241,388225542,154,994-392-3850,176,176,12
67205,train_landmarks/2118949241.parquet,2118949241,388232076,95,https://www.tianjiagenomes.com,310,122,30
67206,train_landmarks/2118949241.parquet,2118949241,388235284,36,90 kerwood circle,45,9,17


In [23]:
((train_df.seqlen>2*train_df.labellen)&(train_df.nonullhand>2*train_df.labellen)).mean()

0.7585852874657779

In [19]:
supp_df["seqlen"] = supp_df.sequence_id.apply(lambda x: seq_length[str(x)] if str(x) in seq_length else 0)
supp_df["nonullhand"] = train_df.sequence_id.apply(lambda x: nonnullhand_length[str(x)] if str(x) in nonnullhand_length else 0)

supp_df["labellen"] = supp_df.phrase.apply(len)

In [22]:
((supp_df.seqlen>2*supp_df.labellen)&(supp_df.nonullhand>2*supp_df.labellen)).mean()

0.5945088560746252

In [10]:
supp_df = supp_df[supp_df.seqlen>supp_df.labellen].reset_index(drop=True)
train_df = train_df[train_df.seqlen>train_df.labellen].reset_index(drop=True)

In [11]:
len(train_df),len(supp_df)

(64868, 51125)

In [13]:
all_df = pd.concat([train_df,supp_df])

In [13]:
all_df = train_df

In [14]:
(all_df.seqlen>384).mean(),(all_df.labellen>45).mean()

(0.015246346426589381, 0.0)

In [15]:
# px.histogram(all_df.sample(1000),x=["seqlen","labellen"])

In [18]:
# a23+1
for fold_idx,(train, test) in enumerate(gkf.split(train_df.sequence_id, groups=train_df.participant_id)):
    valid_seqid = set()
    for idx in test:
        valid_seqid.add(train_df.sequence_id[idx])
    train_info = []
    valid_info = []
    for seqid,label in zip(train_df.sequence_id,train_df.phrase):
        fname = f"{seqid}.npy"
        if fname in usable_names and seqid not in valid_seqid:
            train_info.append((fname,label))
    # for seqid,label in zip(supp_df.sequence_id,supp_df.phrase):
    #     fname = f"{seqid}.npy"
    #     if fname in usable_names:
    #         train_info.append((fname,label))
    train_info = pd.DataFrame(train_info,columns=["files","labels"])

    for seqid,label in zip(train_df.sequence_id,train_df.phrase):
        fname = f"{seqid}.npy"
        if fname in usable_names and seqid in valid_seqid:
            valid_info.append((fname,label))
    valid_info = pd.DataFrame(valid_info,columns=["files","labels"])
    print(len(valid_info),len(train_info))
    train_info.to_csv(f"../dataset/folds/foldtune_train.csv",index=False)
    valid_info.to_csv(f"../dataset/folds/foldtune_valid.csv",index=False)
    break

13053 51815
